# Représentations des données SI-DEP (Système d’Informations de DEPistage)

Santé publique France fournit des données publiques sur les tests COVID-19.

https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/



In [1]:
%matplotlib widget

from functools import partial

import numpy as np
import ipympl
import matplotlib.pyplot as plt
import ipywidgets as widgets

from departements import DEPARTMENTS

In [2]:
from plot_france import plot_france
from plot_1dep import plot_1dep
from plot_faster_dyn import plot_faster_dyn

<ipython-input-2-3d63ae390599>:1: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  from plot_france import plot_france
/home/pierre/.pyenv/versions/3.8.2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Taux de positivité sur 7 jours, France

On trace d'abord le taux de positivité sur 7 jours, c'est-à-dire le pourcentage de tests positifs sur 7 jours par rapport au nombre total de tests sur ces mêmes 7 jours.


In [3]:
def handler_yscale(change, ax):
    new = change["new"]
    if new not in ["linear", "log"]:
        return
    ax.set_yscale(new)
    
    if new == "log":
        ylim = list(ax.get_ylim())
        ylim[0] = 0.8
        ax.set_ylim(ylim)
        ylim = ax.get_ylim()
    
    fig = ax.figure
    fig.canvas.draw()
    fig.canvas.flush_events()

In [4]:
w_yscale = widgets.RadioButtons(
    options=['log', 'linear'],
    value='linear',
    description='y-scale:',
    disabled=False
)
w_yscale

RadioButtons(description='y-scale:', index=1, options=('log', 'linear'), value='linear')

In [5]:
ax = plot_france(yscale=w_yscale.value)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
w_yscale.observe(partial(handler_yscale, ax=ax))

## Taux de positivité et d'incidence pour les départements à forte dynamique épidémique

Il est intéressant d'étudier plus en détail le cas des départements pour lesquels l'épidémie est la plus forte. Ici, on considère les départements métropolitains pour lesquels le taux de positivité sur 7 jours est supérieur à 6 %. La première figure représente le taux de positivité sur 7 jours pour ces départements à forte épidémie.

Sur la figure suivante, les mêmes données sont représentées en terme de taux d'incidence, qui est le nombre de tests positifs sur 7 jours pour 100000 habitants. Cette normalisation du nombre de cas positifs fait que l'incidence dépend directement du nombre de tests. Par exemple, si on ne teste pas, le taux d'incidence est égal à 0 même avec une très forte épidémie. On peut donc interpréter cette quantité comme une borne inférieure du nombre de cas pour 100000 habitants. Le niveau d'incidence égale à 50 est considéré comme un seuil d'alerte (ligne horizontale noire sur la figure). On voit que le département de l'Isère a dépassé ce seuil d'alerte le 3 septembre.

Attention, le taux d'incidence peut baisser juste parce qu'on teste moins. C'est ce qui se passe début septembre pour certains départements (par exemple les Bouches-du-Rhône) avec une diminution du taux d'incidence mais une augmentation du taux de positivité!

In [7]:
plot_faster_dyn()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Taux de positivité pour un département décomposé en classes d'âges

Un des facteurs les plus importants pour prédir la gravité du COVID-19 est l'âge. En dessous de 40 ans, même s'il y a quelques cas graves ou invalidant sur le long terme, la proportion de décés est très faible. Cette proportion augmente ensuite jusqu'à atteindre des valeurs importantes.

On trace le taux de positivité pour un département pour différentes classes d'âges.

Pour augmenter la lisibilité des graphiques, on fait ici le choix de rassembler les classes d'âges 70-79, 80-89 et >=90 en une classe d'âges >=70. Si l'épidémie de COVID-19 se transmet jusqu'à cette classe d'âge, on sait qu'il y a une forte probabilité d'avoir beaucoup de monde aux urgences et un nombre important de décés.


In [8]:
w = widgets.Dropdown(
    options=[(dep, idep) for idep, dep in DEPARTMENTS.items()],
    value="38",
    description='Number:',
)
w

Dropdown(description='Number:', index=38, options=(('Ain', '01'), ('Aisne', '02'), ('Allier', '03'), ('Alpes-d…

In [9]:
w_yscale = widgets.RadioButtons(
    options=['log', 'linear'],
    value='linear',
    description='y-scale:',
    disabled=False
)
w_yscale

RadioButtons(description='y-scale:', index=1, options=('log', 'linear'), value='linear')

In [10]:
ax = plot_1dep(38, yscale=w_yscale.value)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
w_yscale.observe(partial(handler_yscale, ax=ax))

Isère : on observe une accélération très rapide (super exponentielle) de l'épidémie dans les premiers jours de septembre (temps de doublement de moins de 10 jours). 